In [1]:
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
import testjx
import numpy as np
import cvxopt
import cv2
import os 
import random
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt import solvers
import face_recognition
import sklearn.metrics as sm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
from pylab import mpl

import warnings
import matplotlib as mpl
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from collections import Counter
import cv2
import os
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
import preprocessing
from tqdm import tqdm
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import imblearn
from sklearn.metrics import balanced_accuracy_score
sn.set()
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC # SVC
from sklearn import metrics
from sklearn.utils import shuffle
from xgboost import XGBClassifier # XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score,recall_score,cohen_kappa_score,precision_score
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
files = os.listdir('/Users/lvjingzhe/Downloads/celebA/train/')
train = []
    
for i in range(20000):
# for i in range(20000):
    img_ = cv2.imread('/Users/lvjingzhe/Downloads/celebA/train/%d.jpg' %(i))

    train.append(img_)

# files = os.listdir('/Users/lvjingzhe/Downloads/celebA/test/')
# test = []
    
# for i in range(2000,2200):
#     img_ = cv2.imread('/Users/lvjingzhe/Downloads/celebA/test/%d.jpg' %(i))

#     test.append(img_)

In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score

In [5]:
train_image=np.array(train)
train_label=np.concatenate((np.array([0]*19000),np.array([1]*1000)))

In [344]:
model_feat

In [6]:
import time
start=time.time()
base_model= VGG19( weights='imagenet', include_top=False,input_shape=(64,64,3))
x = base_model.output
# x = Dropout(0.2)(x)
predictions = Flatten()(x)

model_feat = Model(inputs=base_model.input,outputs=predictions)

train_features = model_feat.predict(train_image/255)
# test_features=model_feat.predict(test_image/255)
end=time.time()
print('running time:{}'.format(end-start))

W0113 16:38:52.382563 140735694340992 deprecation.py:506] From /Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


running time:275.00307393074036


In [6]:
train_features.shape


(20000, 2048)

In [368]:
'''top_model=Sequential()
top_model.add(Dense(256,activation='relu'))
top_model.add(Dense(2,activation='softmax'))
model=Sequential()
# model.add(model_feat)
model.add(top_model)'''

In [390]:
##VGG19网络结构
'''import tensorflow as tf
from tensorflow import keras
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
]

model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-3),
            loss='categorical_crossentropy'
            )
model.fit(
                        x_tr,y_tr,
                        batch_size=2000,
                        epochs=1000,verbose=0
                       )
'''

In [7]:
x_tr, x_te, y_tr, y_te = train_test_split(train_features,train_label,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 42)

In [884]:
16000/800


20.0

In [8]:
X_test=np.c_[x_te,np.ones(x_te.shape[0])]

In [9]:
# err=(y_tr-sigmoid(np.dot(X_train,w)))**2
# err[index]=2.6*err[index]
def split(x):
    n=y_tr[y_tr==1].shape[0]
    m=int((X_train.shape[0]-n)/19)
    indice=[k*m for k in range(1,19)]
    indice.append(X_train.shape[0]-n)
    x_c=np.concatenate((x[y_tr==0],x[y_tr==1]))
    return np.array(np.split(x_c,indice))
# [np.sum(split(err)[i]) for i in range(20)]

In [10]:
# x_boost=np.concatenate((x_no,x_tr))
# y_boost=np.concatenate((y_no,y_tr))
X_train=np.c_[x_tr,np.ones(x_tr.shape[0])]
n=y_tr[y_tr==1].shape[0]
XX=np.concatenate((X_train[y_tr==0],X_train[y_tr==1]))
YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
m=int((X_train.shape[0]-n)/19)
indice=[k*m for k in range(1,19)]
indice.append(X_train.shape[0]-n)
# X_train=np.c_[x_boost,np.ones(x_boost.shape[0])]
# n=y_boost[y_boost==1].shape[0]
# XX=np.concatenate((X_train[y_boost==0],X_train[y_boost==1]))
# YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
# m=int((X_train.shape[0]-n)/19)
# indice=[k*m for k in range(1,19)]
# indice.append(X_train.shape[0]-n)
pre_XX=np.array(np.split(XX,indice))
pre_YY=np.array(np.split(YY,indice))

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)

In [ ]:
def max_mean_loss(y_true, y_probs,penalty = None):
    n=y_true[y_true==1].shape[0]
    XX=tf.concat((X_train[y_tr==0],X_train[y_tr==1]))
    YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
    m=int((X_train.shape[0]-n)/19)
    indice=[k*m for k in range(1,19)]
    indice.append(X_train.shape[0]-n)
    pre_XX=np.array(np.split(XX,indice))
    pre_YY=np.array(np.split(YY,indice))

    f=tf.reduce_max([tf.reduce_mean((y_probs-y_true)**2 ) for x,y in zip(pre_XX,pre_YY)])


## Traditional classifiers：LR、CS_LR、AUSTBOOST、BAGGING

In [687]:
class LogisticRegression_:
    
    #默认没有正则化，正则项参数默认为1，学习率默认为0.001，迭代次数为10001次
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.1,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))
        
    def loss(self,x,y):
        m=x.shape[0]
        y_pred = self.sigmoid(x.dot(self.W))
        return (-1/m) * np.sum((y*np.log(y_pred) + (1-y)*np.log(1-y_pred)))
    
    def fit(self,x,y):
        lossList = []
        #计算总数据量
        m = x.shape[0]
        #给x添加偏置项
        
        #计算总特征数
        n = x.shape[1]
        #初始化W的值,要变成矩阵形式
        np.random.seed(3421)
        self.W = np.ones((n,1))
        #X转为矩阵形式
       
        #y转为矩阵形式，这步非常重要,且要是m x 1的维度格式
        y_=y.reshape((-1,1))
        #循环epochs次
        for i in range(self.epochs):
            #预测值
            h = self.sigmoid(x.dot(self.W))
            gradient = (x.T).dot(h - y_)/m
            
            
            #加入l1和l2正则项，和之前的线性回归正则化一样
            if self.penalty == 'l2':
                gradient = gradient + self.Lambda * self.W
            elif self.penalty == 'l1':
                gradient = gradient + self.Lambda * np.sign(self.W)
          
            self.W = self.W-self.a * gradient
            if i % 50 == 0:
                lossList.append(self.loss(x,y_))
		#返回系数，和损失列表
        return self.W,lossList

In [12]:
import time
start_i=time.time()
model_in=LogisticRegression()
# model_in.fit(pt.fit_transform(train_image_),train_label_)
model_in.fit(x_tr,y_tr)
end_i=time.time()
y_train_proba=model_in.predict_proba(x_tr)
y_train_label=model_in.predict(x_tr)
y_test_proba=model_in.predict_proba(x_te)
# print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=model_in.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end_i-start_i))
from sklearn.metrics import accuracy_score
# print('Accuracy on training set：{:.2%}'.format(accuracy_score(y_train_label,y_tr)))
# print('Accuracy on testing set：{:.2%}'.format(accuracy_score(y_test_label,y_te)))
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
# print(classification_report(y_tr,y_train_label))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3812
           1       0.80      0.69      0.74       188

    accuracy                           0.98      4000
   macro avg       0.89      0.84      0.86      4000
weighted avg       0.98      0.98      0.98      4000

Running time:3.03 s
Balanced-Accuracy on testing set：84.14%
Recall on testing set：69.15%
F-measure on testing set：70.12%


In [17]:
##SVM
from sklearn.metrics import fbeta_score
import time
start=time.time()
model_svc=SVC(probability=True)

model_svc.fit(x_tr,y_tr)
end=time.time()
y_train_proba=model_svc.predict_proba(x_tr)
y_train_label=model_svc.predict(x_tr)
y_test_proba=model_svc.predict_proba(x_te)

y_test_label=model_svc.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
# from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM：{:.2%}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3812
           1       0.98      0.48      0.65       188

    accuracy                           0.98      4000
   macro avg       0.98      0.74      0.82      4000
weighted avg       0.98      0.98      0.97      4000

Running time:167.80秒
Balanced-Accuracy on testing set：74.18%
Recall on testing set：48.40%
F-measure on testing set：51.08%
Score of SVM：57.31%


In [18]:
from imblearn.over_sampling import RandomOverSampler,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

X_resampled, y_resampled = rus.fit_resample(x_tr,y_tr)

In [23]:
##RUS-SVM
import time
start=time.time()
model_svc_r=SVC(gamma='scale',probability=True)
# model_in.fit(pt.fit_transform(train_image_),train_label_)
model_svc_r.fit(X_resampled, y_resampled)
end=time.time()
y_train_proba=model_svc_r.predict_proba(x_tr)
y_train_label=model_svc_r.predict(x_tr)
y_test_proba=model_svc_r.predict_proba(x_te)
# print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=model_svc_r.predict(x_te)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)

print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




Running time:6.83秒
Balanced-Accuracy on testing set：89.31%
Recall on testing set：90.43%
F-measure on testing set：73.03%
Score of SVM:0.836


In [26]:
##SVM+cost-learning
import time
start=time.time()
model_svc3=SVC(class_weight='balanced',probability=True)
# model_in.fit(pt.fit_transform(train_image_),train_label_)
# model_svc3.fit(X_resampled_smote,y_resampled_smote)
model_svc3.fit(x_tr,y_tr)
end=time.time()
y_train_proba=model_svc3.predict_proba(x_tr)
y_train_label=model_svc3.predict(x_tr)
y_test_proba=model_svc3.predict_proba(x_te)

y_test_label=model_svc3.predict(x_te)

bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :CS-SVM')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier :CS-SVM
Running time:483.40秒
Balanced-Accuracy on testing set：90.13%
Recall on testing set：86.17%
F-measure on testing set：77.65%
Score of SVM:0.838


In [28]:
##RUSBoost
from imblearn.ensemble import RUSBoostClassifier
rusb = RUSBoostClassifier(random_state=0)
start=time.time()
rusb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=rusb.predict_proba(x_tr)
y_train_label=rusb.predict(x_tr)
y_test_proba=rusb.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=rusb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Boost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




AUC:0.9226044294612757
              precision    recall  f1-score   support

           0       0.99      0.89      0.94      3812
           1       0.26      0.78      0.39       188

    accuracy                           0.88      4000
   macro avg       0.62      0.83      0.66      4000
weighted avg       0.95      0.88      0.91      4000

Classifier :RUS-Boost
Running time:8.57秒
Balanced-Accuracy on testing set：83.32%
Recall on testing set：77.66%
F-measure on testing set：64.26%
Score of SVM:0.744


In [29]:
##MLP
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(x_tr,y_tr)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :MLP')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.9706037485209084
Classifier :MLP
Running time:23.58秒
Balanced-Accuracy on testing set：84.09%
Recall on testing set：69.15%
F-measure on testing set：69.96%
Score of SVM:0.737


In [31]:
from imblearn.over_sampling import SMOTE

X_resampled_smote,y_resampled_smote  = SMOTE().fit_resample(x_tr, y_tr)

In [32]:
##MLP+SMOTE
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(X_resampled_smote,y_resampled_smote)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)

y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :SMOTE-MLP')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3812
           1       0.72      0.73      0.73       188

    accuracy                           0.97      4000
   macro avg       0.85      0.86      0.86      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :SMOTE-MLP
Running time:17.00秒
Balanced-Accuracy on testing set：85.99%
Recall on testing set：73.40%
F-measure on testing set：73.24%
Score of SVM:0.768


In [40]:
##MLP+RUS
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=10, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)

y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :RUS-MLP')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier :RUS-MLP
Running time:3.05秒
Balanced-Accuracy on testing set：90.13%
Recall on testing set：92.02%
F-measure on testing set：74.28%
Score of SVM:0.85


In [598]:
from imblearn.under_sampling import NearMiss
# nm1 = NearMiss(version=1)
nm2= NearMiss(version=2)
nm3= NearMiss(version=3)
# X_resampled_1, y_resampled_1 = nm1.fit_resample(x_tr, y_tr)
X_resampled_2, y_resampled_2= nm2.fit_resample(x_tr, y_tr)

In [596]:
from imblearn.over_sampling import ADASYN
X_resampled_,y_resampled_=ADASYN(random_state=42).fit_resample(x_tr,y_tr)

In [41]:
##CART+cost-learing
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(class_weight='balanced',random_state=0)
start=time.time()


clf.fit(x_tr,y_tr)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :CS-Cart')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of CS-Cart:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.6801505882878257
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3812
           1       0.37      0.39      0.38       188

    accuracy                           0.94      4000
   macro avg       0.67      0.68      0.67      4000
weighted avg       0.94      0.94      0.94      4000

Classifier :CS-Cart
Running time:15.70秒
Balanced-Accuracy on testing set：68.02%
Recall on testing set：39.36%
F-measure on testing set：39.08%
Score of SVM:0.484


In [1010]:
(0.39+0.682+0.391)*0.3+0.1/15

0.44556666666666667

In [42]:
##CART+SMOTE
clf = DecisionTreeClassifier(random_state=0)
start=time.time()


clf.fit(X_resampled_smote,y_resampled_smote)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :SMOTE-Cart')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SMOTE-Cart:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.7088589225514055
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3812
           1       0.31      0.47      0.38       188

    accuracy                           0.93      4000
   macro avg       0.64      0.71      0.67      4000
weighted avg       0.94      0.93      0.93      4000

Classifier :SMOTE-Cart
Running time:111.43秒
Balanced-Accuracy on testing set：70.89%
Recall on testing set：46.81%
F-measure on testing set：44.55%
Score of SVM:0.535


In [49]:
##CART+RUS
clf = DecisionTreeClassifier(random_state=0)
start=time.time()


clf.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Cart')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of RUS-Cart:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.7916210845928869
              precision    recall  f1-score   support

           0       0.99      0.77      0.87      3812
           1       0.15      0.81      0.25       188

    accuracy                           0.77      4000
   macro avg       0.57      0.79      0.56      4000
weighted avg       0.95      0.77      0.84      4000

Classifier :RUS-Cart
Running time:1.25秒
Balanced-Accuracy on testing set：79.16%
Recall on testing set：81.38%
F-measure on testing set：55.53%
Score of RUS-Cart:0.721


In [1005]:
(0.555+0.792+0.81)*0.3+0.1/1.22

0.7290672131147541

In [45]:
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
smb = SMOTEBoostClassifier(random_state=0)


In [624]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(x_tr,y_tr)

In [52]:
##SMOTEBoost
start=time.time()
smb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=smb.predict_proba(x_tr)
y_train_label=smb.predict(x_tr)
y_test_proba=smb.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=smb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :SMOTE-Boost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.918534136322029
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      3812
           1       0.46      0.63      0.53       188

    accuracy                           0.95      4000
   macro avg       0.72      0.80      0.75      4000
weighted avg       0.96      0.95      0.95      4000

Classifier :SMOTE-Boost
Running time:234.80秒
Balanced-Accuracy on testing set：79.81%
Recall on testing set：63.30%
F-measure on testing set：60.91%
Score :0.673


In [50]:
adb=AdaBoostClassifier(random_state=0)
start=time.time()
adb.fit(X_resampled, y_resampled )
end=time.time()
y_train_proba=adb.predict_proba(x_tr)
y_train_label=adb.predict(x_tr)
y_test_proba=adb.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=adb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Boost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.9448515047665825
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      3812
           1       0.23      0.90      0.37       188

    accuracy                           0.86      4000
   macro avg       0.61      0.88      0.64      4000
weighted avg       0.96      0.86      0.89      4000

Classifier :RUS-Boost
Running time:6.59秒
Balanced-Accuracy on testing set：87.67%
Recall on testing set：89.89%
F-measure on testing set：69.38%
Score :0.816


In [51]:
##ADABoost
adb=AdaBoostClassifier(random_state=0)
start=time.time()
adb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=adb.predict_proba(x_tr)
y_train_label=adb.predict(x_tr)
y_test_proba=adb.predict_proba(x_te)

y_test_label=adb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))

print('Classifier :Adaboost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3812
           1       0.73      0.55      0.63       188

    accuracy                           0.97      4000
   macro avg       0.85      0.77      0.81      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :Adaboost
Running time:81.01秒
Balanced-Accuracy on testing set：76.90%
Recall on testing set：54.79%
F-measure on testing set：56.25%
Score:0.62


In [54]:
##BAgging
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(
                                random_state=0)


start=time.time()


bag.fit(x_tr,y_tr)
end=time.time()
y_train_proba=bag.predict_proba(x_tr)
y_train_label=bag.predict(x_tr)
y_test_proba=bag.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=bag.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :Bagging')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




AUC:0.872678104976446
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3812
           1       0.84      0.44      0.58       188

    accuracy                           0.97      4000
   macro avg       0.91      0.72      0.78      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :Bagging
Running time:285.68秒
Balanced-Accuracy on testing set：71.86%
Recall on testing set：44.15%
F-measure on testing set：46.43%
Score:0.536


In [56]:
##Bagging+RUS
bag = BaggingClassifier(
                                random_state=0)


start=time.time()


bag.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=bag.predict_proba(x_tr)
y_train_label=bag.predict(x_tr)
y_test_proba=bag.predict_proba(x_te)

y_test_label=bag.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :RUS-Bag')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.99      0.86      0.92      3812
           1       0.23      0.85      0.36       188

    accuracy                           0.86      4000
   macro avg       0.61      0.85      0.64      4000
weighted avg       0.96      0.86      0.90      4000

Classifier :RUS-Bag
Running time:8.17秒
Balanced-Accuracy on testing set：85.40%
Recall on testing set：84.57%
F-measure on testing set：66.41%
Score:0.781


In [1113]:
##Bagging +SMOte
'''bag = BaggingClassifier(
                                random_state=0)


start=time.time()


bag.fit(X_resampled_smote,y_resampled_smote)
end=time.time()
y_train_proba=bag.predict_proba(x_tr)
y_train_label=bag.predict(x_tr)
y_test_proba=bag.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=bag.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :SMOTE-Bag')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))'''



AUC:0.8955579803978477
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      3812
           1       0.56      0.44      0.49       188

    accuracy                           0.96      4000
   macro avg       0.77      0.71      0.74      4000
weighted avg       0.95      0.96      0.96      4000

运行时间:613.53秒
训练集准确率：99.91%
测试集准确率：95.75%
测试集Bacc：71.22%
测试集f2：45.14%
测试集recall：44.15%


In [1111]:
(0.44+0.464+0.719)*0.33+0.01/281

0.5356255871886121

In [821]:
split(err)[0].shape

(799,)

In [94]:
## XUAN[2019]+L2正则项
def split(x):
    n=y_tr[y_tr==1].shape[0]
    m=int((X_train.shape[0]-n)/19)
    indice=[k*m for k in range(1,19)]
    indice.append(X_train.shape[0]-n)
    x_c=np.concatenate((x[y_tr==0],x[y_tr==1]))
    return np.array(np.split(x_c,indice))

class solution_XX:
    
    #默认没有正则化，正则项参数默认为1，学习率默认为0.001，迭代次数为10001次
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.5,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))
        
#     err=(y_tr-s.sigmoid(np.dot(X_train,w)))**2
#     err[index]=2.6*err[index]

    def f_XX(self,X,Y):
        if self.penalty=='l1':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(np.abs(self.W)) for x,y in zip(X,Y)])
        elif self.penalty=='l2':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(self.W**2) for x,y in zip(X,Y)])#pre_Xtrain,pre_Ytrain
        else:f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 ) for x,y in zip(X,Y)])

        return f         
    def Gf_XX(self,X,Y):#求导数矩阵N*2
        if self.penalty=='l1':d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+self.Lambda*np.sign(self.W )for x,y in zip(X,Y)])
        elif self.penalty=='l2':
            d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+2*self.Lambda*self.W for x,y in zip(X,Y)])
#     return d.reshape(20,12289)
        else:d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W)))) for x,y in zip(X,Y)])
        return d
    def direction_XX(self,X,Y):
        gra=self.Gf_XX(X,Y)#导数矩阵N*30001
        p=matrix(gra.dot(gra.T),tc='d')
        q=matrix(-self.f_XX(X,Y),tc='d')#N维列向量
        G=matrix(np.diag(np.array([-1]*20)),tc='d')#N=20
        h=matrix(np.array([[0]]*20),tc='d')
        A=matrix([[1.0]]*20)
        b=matrix([1.0])
        solvers.options['show_progress'] = False
        sol = solvers.qp(p,q,G,h,A,b)
        t=np.array(sol['x'])
        d= -(gra.T.dot(t))
        return d.reshape((X_train.shape[-1],))#30001维列向量
# def BB_X(v):
#     return sigmoid(np.dot(pre_Xtest,v))
# def ff_1_X(w,v):
#     w=w.reshape(15,1)
#     return np.mean((sigmoid(np.dot(BB(v),w[:-1])+w[-1])-test_label)**2 )
# def ff_X(w):
#     return np.mean((sigmoid(np.dot(pre_Xtest,w))-test_label)**2 )
    def fit(self,X,Y):
        
        call=[]
        pre=[]
        loss=[]
        testloss=[]
        np.random.seed(1324)
        self.W=np.random.random((X_train.shape[-1],))*2-1
#         self.W=w_2
        n=y_te[y_te==1.].shape[0]
        for k in range(200):
    #     while np.linalg.norm(d)//10**(-8) >= 10:
            d=self.direction_XX(X,Y)
#             print(np.linalg.norm(d))
            if np.linalg.norm(d)//10**(-7) < 25:
                break
            sigma=0.8
            f_1=np.max(self.f_XX(X,Y))
            w=self.W
            self.W=d*sigma+w
            while np.max(self.f_XX(X,Y))>np.max(f_1):
                sigma=sigma*0.8
                self.W=d*sigma+w
            self.W=d*sigma+w

            
    #         output=output.reshape(4000,)
    #         pt=max(output[output==1.].shape[0],1)
    #         m=0
    #         for j in range(4000):
    #             if output[j]==test_label[j]==1:
    #                 m+=1


    # # b1=np.random.random((1,10))*2-1


            
    #         call.append(m/n)
    #         pre.append(m/pt)
    #         loss.append(max(f_X(w)))
    #         testloss.append(ff_X(w))
        return self.W

In [23]:
s_2=solution_XX(penalty='l2')
start_x=time.time()
w_2 = s_2.fit(pre_XX,pre_YY)
end_x=time.time()
y_pred=s_2.sigmoid(X_test.dot(w_2))

y_train_pred=s_2.sigmoid(X_train.dot(w_2))
y_train_pred[y_train_pred>=0.5]=1
y_train_pred[y_train_pred<0.5]=0
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0
# # C_x.append(precision_score(y_pred,train_label_[test_index]))
bacc=balanced_accuracy_score(y_te,y_pred)
fbeta=fbeta_score(y_te,y_pred,beta=2.94)
recall=recall_score(y_te,y_pred)
print('Classifier :L2 max-mean loss')
print("Running time:%.2f s"%(end_x-start_x))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_pred)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_pred)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_pred,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))



Classifier :L2 max-mean loss
Running time:20.09 s
Balanced-Accuracy on testing set：90.07%
Recall on testing set：86.70%
F-measure on testing set：77.13%
Score:0.838


In [68]:
s=solution_XX()
start=time.time()
w = s.fit(pre_XX,pre_YY)
end=time.time()
y_pred=s.sigmoid(X_test.dot(w))

y_train_pred=s.sigmoid(X_train.dot(w))
y_train_pred[y_train_pred>=0.5]=1
y_train_pred[y_train_pred<0.5]=0
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0
# # C_x.append(precision_score(y_pred,train_label_[test_index]))
bacc=balanced_accuracy_score(y_te,y_pred)
fbeta=fbeta_score(y_te,y_pred,beta=2.94)
recall=recall_score(y_te,y_pred)
print('Classifier : max-mean loss')
print("Running time:%.2f"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_pred)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_pred)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_pred,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier : max-mean loss
Running time:13.82
Balanced-Accuracy on testing set：88.20%
Recall on testing set：83.51%
F-measure on testing set：73.75%
Score:0.811


## Mean_uncertain method

In [24]:
def sigmoid_(x,u):
    return 1.0/(1.0+np.exp(-x-u))

In [25]:
pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
n_len=pre_in[pre_in>=0].shape[0]
index=np.argwhere(pre_in>=0).reshape(n_len,)

In [26]:
def meanuncertainty(x,n):
    r=[]
    for i in range(0,len(x)+1-n,n//5):
        r.append(np.mean(x[i:i+n]))
    return min(r),max(r)

In [30]:
def equa_xh(x,n):
    pre_in=np.dot(X_train,w_2).reshape(x_tr.shape[0],)
    ini_err=y_tr-sigmoid_(pre_in,x)
    err_w=shuffle(np.concatenate((np.random.choice(ini_err[y_tr==0],ini_err[y_tr==1].shape[0]),ini_err[y_tr==1])))
    for k in range(ini_err.shape[0]):
    
        if y_tr[k]==1:
            ini_err[k]=10*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]

In [28]:
def equa_m(x,n):
    
    ini_err=y_tr-sigmoid_(pre_in,x)
#     err_w=shuffle(np.concatenate((np.random.choice(ini_err[y_tr==0],ini_err[y_tr==1].shape[0]),ini_err[y_tr==1])))
    for k in range(ini_err.shape[0]):
    
        if y_tr[k]==1:
            ini_err[k]=10*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]


In [37]:
fsolve(lambda x:equa_xh(x,560),0.5 )

array([0.08102938])

In [39]:
meanxh_1=np.array([fsolve(lambda x:equa_xh(x,n),0.5 ) for n in range(10,1000,10)])
# meanxh_2=np.array([fsolve(lambda x:equa_xh(x,n),0.5 ) for n in range(1000,10000,100)])

In [44]:
pd.DataFrame(meanxh_1,index=np.array(range(10,1000,10))).to_csv('glass_upper_mean.csv',index=False)

In [940]:
mean_xh=np.concatenate((meanxh_1,meanxh_2))[:-35].reshape(-1)

In [880]:
meanuncertainty(y_tr-sigmoid_(pre_in,0),16000)

(0.0004772297745227459, 0.0004772297745227459)

In [204]:
mean_1=np.array([fsolve(lambda x:equa_m(x,n),0.5 ) for n in range(10,1000,10)])
mean_2=np.array([fsolve(lambda x:equa_m(x,n),0.5 ) for n in range(1000,10000,100)])

In [282]:

mean_lr=np.concatenate((mean_1,mean_2))[:-20].reshape(-1)


In [966]:
12800/650

19.692307692307693

## Choice optimal window size on  Cross-validation set

In [101]:
import imblearn
from sklearn.metrics import balanced_accuracy_score
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_modelxh(X,y,m,model=solution_XX()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=1, random_state=1)
    for train_index,test_index in cv.split(X,y):
        
#         N=X[train_index].shape[0]
#         n=y[train_index][y[train_index]==1].shape[0]
#         c_1=np.concatenate((X[train_index][y[train_index]==0],X[train_index][y[train_index]==1]))
#         l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
#         M=int((N-n)/19)
#         indice=[k*M for k in range(1,19)]
#         indice.append(N-n)
#         pre_cv=np.array(np.split(c_1,indice))
#         pre_y=np.array(np.split(l_1,indice))
#         start=time.time()
#         w= model.fit(pre_cv,pre_y)
#         end=time.time()
#         print(end-start)
#         w_x.append(w)
        prob_y=sigmoid_(X_train[test_index].dot(w_2),m)
        
        prdict_y=np.round(prob_y)
        rec.append(recall_score(y[test_index],prdict_y))
        pre.append(precision_score(y[test_index],prdict_y))

        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        f2.append(fbeta_score(y[test_index],prdict_y,beta=2.94))
 
    return bacc,f2,rec,pre
    
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
           
               

In [55]:
meanxh_1=meanxh_1.reshape(99,)

In [88]:
meanxh_1[1]

2.436972672117328

In [96]:
start=time.time()
evaluate_modelxh(X_train,y_tr,meanxh_1[1])
end=time.time()
print(end-start)

11.566361665725708
10.885604858398438
10.936495780944824
11.015794277191162
11.218694925308228
60.134746074676514


In [97]:

all_resultsxh = []

for k in meanxh_1:
    result=np.mean(evaluate_modelxh(X_train,y_tr,k),axis=1)
    metric_res = {'upper_mean': k}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxh.append(metric_res)

11.73898983001709
11.121030807495117
11.174737930297852
12.029088020324707
13.282639980316162
12.631754875183105
12.181988954544067
10.262044906616211
11.65503215789795
11.197637796401978
12.983259916305542
12.804320096969604
11.872678995132446
12.662386894226074
12.476053953170776
12.269008874893188
11.643685817718506
11.485091924667358
11.457480192184448
11.54461407661438
11.973052024841309
10.619282007217407
11.33933711051941
11.54064393043518
11.010878086090088
11.210134029388428
11.531489133834839
12.165132999420166
12.23213815689087
11.158826112747192
11.2482008934021
12.193104028701782
11.94871997833252
11.479596138000488
11.426672220230103
11.367796897888184
10.417709112167358
11.163100004196167
11.110937118530273
11.550012111663818
11.526576042175293
11.59375786781311
11.157629013061523
12.361654996871948
11.655821084976196
11.971081972122192
11.351592063903809
11.106999158859253
11.659721851348877
11.253346920013428
11.258503913879395
11.255056858062744
11.136001825332642
11.

In [102]:

all_resultsxh_w = []

for k in meanxh_1:
    result=np.mean(evaluate_modelxh(X_train,y_tr,k),axis=1)
    metric_res = {'upper_mean': k}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxh_w.append(metric_res)

In [104]:
pd.DataFrame(all_resultsxh_w).to_csv('CV_select_mu_MM_gla_simple.csv')

In [270]:
import imblearn
from sklearn.metrics import balanced_accuracy_score
metrics_names=['Balanced_acc','F2_score','AUC','Recall','Precision']
def evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,auc,rec,pre=list(),list(),list(),list(),list()
   
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=1, random_state=1)
    for train_index,test_index in cv.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_),m)
                auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,auc,rec,pre

In [285]:
##交叉验证集选上下均值
all_results = []

for k in mean_lr:
    result=np.mean(evaluate_model(x_tr,y_tr,k),axis=1)
    metric_res = {'upper_mean': k}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_results.append(metric_res)

In [286]:

pd.DataFrame(all_results).to_csv('CV_which_mu_to_set_inLRBIAS.csv')


In [3]:
'''def huber(true, pred, delta):
    loss = np.where(np.abs(true-pred) < delta , 0.5*((true-pred)**2), delta*np.abs(true - pred) - 0.5*(delta**2))
    return np.sum(loss)'''

'def huber(true, pred, delta):\n    loss = np.where(np.abs(true-pred) < delta , 0.5*((true-pred)**2), delta*np.abs(true - pred) - 0.5*(delta**2))\n    return np.sum(loss)'

## Volatility-uncertain method


In [67]:
def F_u(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[1]*st.norm.cdf(c/x[1])/(x[0]+x[1]))
        else:
            p.append(1-2*x[0]*st.norm.cdf(-c/x[0])/(x[0]+x[1]))
    return np.array(p)
def F_L(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[0]*st.norm.cdf(c/x[0])/(x[0]+x[1]))
        else:
            p.append(1-2*x[1]*st.norm.cdf(-c/x[1])/(x[0]+x[1]))
    return np.array(p)

In [873]:
def equa_v(x,n):
 
    
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
#     for k in range(y_tr.shape[0]):
    
#         if y_tr[k]==1:
#             err_u[k]=2*err_u[k]
#             err_L[k]=19*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

In [1084]:
def equa_vxh(x,n):
    pre_in=np.dot(X_train,w).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
#     for k in range(y_tr.shape[0]):
    
#         if y_tr[k]==1:
#             err_u[k]=2*err_u[k]
#             err_L[k]=19*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

In [1094]:
# var_1=np.array([fsolve(lambda x:equa_vxh(x,n),[0.5,1.5] ) for n in range(50,110,10)])
var_2=np.array([fsolve(lambda x:equa_vxh(x,n),[0.5,1.5] ) for n in range(500,1000,50)])
var_3=np.array([fsolve(lambda x:equa_vxh(x,n),[0.5,1.5] ) for n in range(1000,6000,100)])



In [1096]:
var_xh=np.concatenate((var_2,var_3))
var_xhmean=pd.DataFrame(var_xh,index=np.hstack((
                                           np.array(range(500,1000,50)),np.array(range(1000,6000,100)))))
var_xhmean.to_csv('var_xhmean_celeb.csv')

## Choice optimal window size on  Cross-validation set

In [82]:
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_varxh(X,y,m,model=solution_XX(penalty='l2')):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=1, random_state=1)
    for train_index,test_index in cv.split(X,y):
        
        N=X[train_index].shape[0]
        n=y[train_index][y[train_index]==1].shape[0]
        c_1=np.concatenate((X[train_index][y[train_index]==0],X[train_index][y[train_index]==1]))
        l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
        M=int((N-n)/19)
        indice=[k*M for k in range(1,19)]
        indice.append(N-n)
        pre_cv=np.array(np.split(c_1,indice))
        pre_y=np.array(np.split(l_1,indice))
        w= model.fit(pre_cv,pre_y)
#         w_x.append(w)
        prob_y=F_u(m,X_train[test_index].dot(w))
        
        prdict_y=np.round(prob_y)
        rec.append(recall_score(y[test_index],prdict_y))
        pre.append(precision_score(y[test_index],prdict_y))

        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        f2.append(fbeta_score(y[test_index],prdict_y,beta=2.94))
 
    return bacc,f2,rec,pre

In [85]:
for train_index,test_index in cv.split(X_train,y_tr):
        
        N=X_train[train_index].shape[0]
        n=y_tr[train_index][y_tr[train_index]==1].shape[0]
        c_1=np.concatenate((X_train[train_index][y_tr[train_index]==0],X_train[train_index][y_tr[train_index]==1]))
        l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
        M=int((N-n)/19)
        indice=[k*M for k in range(1,19)]
        indice.append(N-n)
        pre_cv=np.array(np.split(c_1,indice))
        pre_y=np.array(np.split(l_1,indice))

In [1104]:

all_resultsxhvar = []

for k in var_xh:
    result=np.mean(evaluate_varxh(X_train,y_tr,k),axis=1)
    metric_res = {'lower-var': k[0],'upper-var':k[1]}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxhvar.append(metric_res)

In [1105]:
pd.DataFrame(all_resultsxhvar).to_csv('CV_which_sigma_to_set_inxhBIAS.csv')

In [ ]:
var_1=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(50,110,10)])
var_2=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(110,1000,50)])
var_3=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(1000,6000,100)])


                

In [566]:
var=np.concatenate((var_1,var_2,var_3))

In [571]:
var_mean=pd.DataFrame(var,index=np.hstack((np.array(range(50,110,10)),
                                           np.array(range(110,1000,50)),np.array(range(1000,6000,100)))))

In [573]:
var_mean.to_csv('var_mean_celeb.csv')

In [576]:
def var_evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,auc,rec,pre=list(),list(),list(),list(),list()
   
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=1, random_state=1)
    for train_index,test_index in cv.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=F_u(m,(np.dot(X[test_index],model_in.coef_.T)+model_in.intercept_).reshape(X[test_index].shape[0],))
        
                auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,auc,rec,pre

In [585]:

all_results_var = []

for k in var:
    result=np.mean(var_evaluate_model(x_tr,y_tr,k),axis=1)
    metric_res = {'lower_var':k[0],'upper_var': k[1]}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_results_var.append(metric_res)

In [587]:
pd.DataFrame(all_results_var).to_csv('CV_which_sigma_to_set_inLRBIAS.csv')

## testing performance

In [86]:
##volatility-bias max-mean

y_preX_v=F_u([0.075,0.487],np.dot(X_test,w_2)).reshape(x_te.shape[0],)

y_preX_v[y_preX_v<0.5]=0
y_preX_v[y_preX_v>=0.5]=1
print(classification_report(y_te,y_preX_v))
bacc=balanced_accuracy_score(y_te,y_preX_v)
fbeta=fbeta_score(y_te,y_preX_v,beta=2.94)
recall=recall_score(y_te,y_preX_v)
print('Classifier : volatility-unceratin max-mean ')
print("Running time:%.2f"%(end_x-start_x))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_v)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_v)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_v,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))



              precision    recall  f1-score   support

           0       1.00      0.90      0.95      3812
           1       0.32      0.92      0.47       188

    accuracy                           0.90      4000
   macro avg       0.66      0.91      0.71      4000
weighted avg       0.96      0.90      0.92      4000

Classifier : volatility-unceratin max-mean 
Running time:20.36
Balanced-Accuracy on testing set：91.16%
Recall on testing set：92.02%
F-measure on testing set：76.95%
Score:0.859


In [92]:
y_preX_v=F_u([0.26,2.27],np.dot(x_te,model_in.coef_.T)+model_in.intercept_).reshape(x_te.shape[0],)

y_preX_v[y_preX_v<0.5]=0
y_preX_v[y_preX_v>=0.5]=1
print(classification_report(y_te,y_preX_v))
bacc=balanced_accuracy_score(y_te,y_preX_v)
fbeta=fbeta_score(y_te,y_preX_v,beta=2.94)
recall=recall_score(y_te,y_preX_v)
print('Classifier : volatility-unceratin LR ')
print("Running time:%.2f"%(end_i-start_i))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_v)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_v)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_v,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_i-start_i)))



              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3812
           1       0.65      0.78      0.71       188

    accuracy                           0.97      4000
   macro avg       0.82      0.88      0.85      4000
weighted avg       0.97      0.97      0.97      4000

Classifier : volatility-unceratin LR 
Running time:3.26
Balanced-Accuracy on testing set：88.07%
Recall on testing set：78.19%
F-measure on testing set：76.63%
Score:0.805


In [88]:
##Mean-bias LR
y_preX_m=sigmoid_((np.dot(x_te,model_in.coef_.T)+model_in.intercept_).reshape(x_te.shape[0],),4.13)

y_preX_m[y_preX_m<0.5]=0
y_preX_m[y_preX_m>=0.5]=1
print(classification_report(y_te,y_preX_m))
print('Classifier : mean-unceratin LR ')
print("Running time:%.2f"%(end_i-start_i))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_m)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_m)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_m,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_i-start_i)))


              precision    recall  f1-score   support

           0       1.00      0.90      0.94      3812
           1       0.30      0.92      0.45       188

    accuracy                           0.90      4000
   macro avg       0.65      0.91      0.70      4000
weighted avg       0.96      0.90      0.92      4000

Classifier : mean-unceratin LR 
Running time:3.26
Balanced-Accuracy on testing set：90.76%
Recall on testing set：92.02%
F-measure on testing set：75.90%
Score:0.862


In [105]:
##Mean-bias max-mean
y_preX_x=sigmoid_(np.dot(X_test,w_2).reshape(x_te.shape[0],),0.28)

y_preX_x[y_preX_x<0.5]=0
y_preX_x[y_preX_x>=0.5]=1
print(classification_report(y_te,y_preX_x))
bacc=balanced_accuracy_score(y_te,y_preX_x)
fbeta=fbeta_score(y_te,y_preX_x,beta=2.94)
recall=recall_score(y_te,y_preX_x)
print('Classifier : mean-unceratin max-mean ')
print("Running time:%.2f s"%(end_x-start_x))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_x)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_x)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_x,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95      3812
           1       0.32      0.92      0.47       188

    accuracy                           0.90      4000
   macro avg       0.66      0.91      0.71      4000
weighted avg       0.96      0.90      0.92      4000

Classifier : mean-unceratin max-mean 
Running time:20.09 s
Balanced-Accuracy on testing set：91.14%
Recall on testing set：92.02%
F-measure on testing set：76.92%
Score:0.859
